### Setup and Imports ###

In [53]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt

In [54]:
zip_path = r"C:\Users\akshay\Downloads\archive (1).zip"
extract_to = r"C:\Users\akshay\Downloads\gestures"

In [55]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)  

In [56]:
train_dir = r"C:\Users\akshay\Downloads\gestures\train"

In [57]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [58]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

In [59]:
train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 14400 images belonging to 1 classes.


In [60]:
val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3600 images belonging to 1 classes.


### CNN Model Architecture ###

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [62]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(20, activation='softmax')  
])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       1,605,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 20)                  │           2,580 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,627,732 (6.21 MB)

 Trainable params: 1,627,732 (6.21 MB)

 Non-trainable params: 0 (0.00 B)

### Compile and Train ###

In [63]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  
    metrics=['accuracy']
)

In [64]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [65]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=1,
    callbacks=callbacks
)

450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.9870 - loss: 0.0435  

450/450 ━━━━━━━━━━━━━━━━━━━━ 201s 442ms/step - accuracy: 0.9870 - loss: 0.0434 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [66]:
model.save("gesture_model.keras")  

In [67]:
from tensorflow.keras.models import load_model
model = load_model("gesture_model.keras")

In [68]:
print(history.history)

{'accuracy': [0.9980555772781372], 'loss': [0.006541525013744831], 'val_accuracy': [1.0], 'val_loss': [0.0]}


In [69]:
acc = history.history['accuracy'][0]
val_acc = history.history['val_accuracy'][0]
loss = history.history['loss'][0]
val_loss = history.history['val_loss'][0]

In [70]:
print(f"Train Accuracy: {acc:.4f}")
print(f"Val Accuracy: {val_acc:.4f}")
print(f"Train Loss: {loss:.4f}")
print(f"Val Loss: {val_loss:.4f}")

Train Accuracy: 0.9981
Val Accuracy: 1.0000
Train Loss: 0.0065
Val Loss: 0.0000


In [71]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [72]:
model = load_model("gesture_model.keras")

In [73]:
class_labels = [str(i) for i in range(20)] 

In [74]:
folder_path = r"C:\Users\akshay\Downloads\gestures\train"

In [75]:
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

In [76]:
for file_name in image_files:
    image_path = os.path.join(folder_path, file_name)
    img = cv2.imread(image_path)

    if img is None:
        print(f"⚠️ Could not read image: {file_name}")
        continue

    img_resized = cv2.resize(img, image_size)
    img_input = np.expand_dims(img_resized / 255.0, axis=0)

    predictions = model.predict(img_input)
    class_index = np.argmax(predictions[0])
    confidence = predictions[0][class_index]

In [78]:
print(f"🖼️ {image_files} → Prediction: {class_labels[class_index]} ({confidence * 100:.2f}%)")

🖼️ [] → Prediction: 1 (100.00%)
